In [1]:
import sys
sys.path.append("/Users/rq.aita/Scripts")

import pandas as pd
import numpy as np
import hydrology as hydro

# Exercício 5

Calcule novamente a evapotranspiração potencial mensal e anual do exercício resolvido na aula  09, utilizando as equações de Serra, dadas a seguir, para a avaliação dos parâmetros $I$ e $a$ do método de Thornthwaite.
$$
    I = \sum_{i=1}^{12} \left( 0,09 t_i^{\frac{3}{2}} \right) \ \ \ e \ \ \ a = 0,016I + 0,5
$$
Comparar os resultados obtidos em sala e os obtidos por você na solução do exercício. Procure explicar as diferenças encontradas (exercício adaptado de Úmbria e Zandonai, 1996). 

| Mês       | T(°C) | N(h) | Mês      | T(°C) | N(h) |
|-----------|-------|------|----------|-------|------|
| Janeiro   | 23,9  | 13,3 | Julho    | 16,9  | 10,9 |
| Fevereiro | 23,5  | 12,7 | Agosto   | 18,5  | 11,4 |
| Março     | 22,4  | 11,6 | Setembro | 20,3  | 12,0 |
| Abril     | 20,8  | 11,6 | Outubro  | 20,8  | 12,7 |
| Maio      | 17,0  | 11,0 | Novembro | 23,1  | 13,2 |
| Junho     | 16,0  | 10,7 | Dezembro | 23,0  | 13,6 |

## Entradas

In [3]:
e_05_T = np.array([
    23.9, 23.5, 22.4, 20.8, 17. , 16. , 16.9, 18.5, 20.3, 20.8, 23.1, 23. ,
])
e_05_N = np.array([
    13.3, 12.7, 11.6, 11.6, 11. , 10.7, 10.9, 11.4, 12. , 12.7, 13.2, 13.6,
])

## Resolução

In [4]:
e_05_EVTp, e_05_I, e_05_a = hydro.thornthwaite(e_05_T, e_05_N, serra=True)

In [5]:
e_05_data = {
    "T (°C)" : e_05_T,
    "N (h)" : e_05_N,
    "EVTp (mm)" : e_05_EVTp.round(1),
}
e_05_data = pd.DataFrame(
    data=e_05_data, index=["Jan", "Fev", "Mar", "Abr", "Mai", "Jun",
                           "Jul", "Ago", "Set", "Out", "Nov", "Dez"]
)

In [28]:
print(e_05_data)
print("a =", e_05_a.round(2))
print("I =", e_05_I.round(2))

     T (°C)     N  EVTp (mm)
Jan    23.9  13.3      114.8
Fev    23.5  12.7       95.5
Mar    22.4  11.6       87.3
Abr    20.8  11.6       72.2
Mai    17.0  11.0       46.2
Jun    16.0  10.7       38.2
Jul    16.9  10.9       45.2
Ago    18.5  11.4       57.2
Set    20.3  12.0       70.9
Out    20.8  12.7       81.7
Nov    23.1  13.2      102.6
Dez    23.0  13.6      108.2
a = 2.12
I = 101.02


# Exercício 6

Refaça o exemplo da aula 09 (método de Thornthwaite), substituindo a temperatura média mensal em cada mês pela temperatura média anual. Compare os resultados btidos com o exemplo resolvido na referida aula, relativamente às evapotranspirações médias mensais e a evapotranspiração média anual. 

## Entradas

In [16]:
e_06_T = np.array([
    23.9, 23.5, 22.4, 20.8, 17. , 16. , 16.9, 18.5, 20.3, 20.8, 23.1, 23. ,
])
e_06_T = e_06_T.mean() * np.ones(12)

e_06_N = np.array([
    13.3, 12.7, 11.6, 11.6, 11. , 10.7, 10.9, 11.4, 12. , 12.7, 13.2, 13.6,
])

## Resolução

In [29]:
e_06_EVTp, e_06_I, e_06_a = hydro.thornthwaite(e_06_T, e_06_N)

In [30]:
e_06_data = {
    "T (°C)" : e_06_T.round(1),
    "N (h)" : e_06_N,
    "EVTp (mm)" : e_06_EVTp.round(1),
}
e_06_data = pd.DataFrame(
    data=e_06_data, index=["Jan", "Fev", "Mar", "Abr", "Mai", "Jun",
                           "Jul", "Ago", "Set", "Out", "Nov", "Dez"]
)

In [31]:
e_06_data

,T (°C),N (h),EVTp (mm)
Jan,20.5,13.3,88.4
Fev,20.5,12.7,76.2
Mar,20.5,11.6,77.1
Abr,20.5,11.6,74.6
Mai,20.5,11.0,73.1
Jun,20.5,10.7,68.8
Jul,20.5,10.9,72.5
Ago,20.5,11.4,75.8
Set,20.5,12.0,77.2
Out,20.5,12.7,84.4


In [32]:
print("a =", e_06_a.round(2))
print("I =", e_06_I.round(2))

a = 2.23
I = 101.73
